In [1]:
import pandas as pd
import numpy as np

# import BERT tokenization

In [2]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from bert.tokenization import FullTokenizer

2023-09-26 14:47:31.803910: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-26 14:47:31.991247: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-26 14:47:31.992762: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 14:47:33.003904: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
df = pd.read_csv("Data/preparation.csv")
df = df[['Description'	,'product_category_tree_target']]

In [5]:
df.head(3)

,Description,product_category_tree_target
0,Key Features of Elegance Polyester Multicolor ...,Home Furnishing
1,Specifications of Sathiyas Cotton Bath Towel (...,Baby Care
2,Key Features of Eurospa Cotton Terry Face Towe...,Baby Care


# Split data

In [6]:
# Define the ratio for the split
split_ratio = 0.8  # 80% training, 20% testing

# Calculate the number of rows for training and testing
total_rows = df.shape[0]
train_rows = int(split_ratio * total_rows)
test_rows = total_rows - train_rows

# Split the DataFrame into training and testing sets
train_data = df.iloc[:train_rows, :]
test_data = df.iloc[train_rows:, :]


# Label encoding of labels

In [7]:
label = preprocessing.LabelEncoder()
y = label.fit_transform(train_data['product_category_tree_target'])
y = to_categorical(y)
print(y[:5])

[[0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]]


# Build a BERT layer

In [8]:
m_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(m_url, trainable=True)

# Encoding the text

In [9]:


#https://stackoverflow.com/questions/67043468/unparsedflagaccesserror-trying-to-access-flag-preserve-unused-tokens-before-f
# Set the preserve_unused_tokens flag to True
import sys
from absl import flags
sys.argv=['preserve_unused_tokens=False']
flags.FLAGS(sys.argv)

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
        
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len-len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
        
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [10]:
labels = label.classes_
print(labels)

['Baby Care' 'Beauty and Personal Care' 'Computers'
 'Home Decor & Festive Needs' 'Home Furnishing' 'Kitchen & Dining'
 'Watches']


In [11]:
len_cat = len(labels)

In [12]:
def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    
    clf_output = sequence_output[:, 0, :]
    
    lay = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    lay = tf.keras.layers.Dense(32, activation='relu')(lay)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    out = tf.keras.layers.Dense(len_cat, activation='softmax')(lay)  # len_cart = N de cat.
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=2e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [13]:
max_len = 250
train_input = bert_encode(train_data.Description.values, tokenizer, max_len=max_len)
test_input = bert_encode(test_data.Description.values, tokenizer, max_len=max_len)
train_labels = y

In [14]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_word_ids (InputLayer  [(None, 250)]                0         []                            
 )                                                                                                
                                                                                                  
 input_mask (InputLayer)     [(None, 250)]                0         []                            
                                                                                                  
 segment_ids (InputLayer)    [(None, 250)]                0         []                            
                                                                                                  
 keras_layer (KerasLayer)    [(None, 768),                1094822   ['input_word_ids[0][0]',  

In [15]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

train_sh = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=3,
    
    callbacks=[checkpoint, earlystopping],
    batch_size=32,
    verbose=1
)

Epoch 1/3
21/21 [==============================] - ETA: 0s - loss: 2.1019 - accuracy: 0.1429 
Epoch 1: val_accuracy improved from -inf to 0.13095, saving model to model.h5


/home/apprenant/Documents/Developper_modele_deep_learning_multi-inputs/myenv/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


21/21 [==============================] - 672s 31s/step - loss: 2.1019 - accuracy: 0.1429 - val_loss: 2.0335 - val_accuracy: 0.1310
Epoch 2/3
21/21 [==============================] - ETA: 0s - loss: 1.9328 - accuracy: 0.1592 
Epoch 2: val_accuracy did not improve from 0.13095
21/21 [==============================] - 680s 33s/step - loss: 1.9328 - accuracy: 0.1592 - val_loss: 2.0494 - val_accuracy: 0.0952
Epoch 3/3
21/21 [==============================] - ETA: 0s - loss: 1.9478 - accuracy: 0.1994 
Epoch 3: val_accuracy did not improve from 0.13095
21/21 [==============================] - 657s 31s/step - loss: 1.9478 - accuracy: 0.1994 - val_loss: 1.9589 - val_accuracy: 0.0655
